In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split # Для разделения данных на обучающую и тестовую выборку
from sklearn.feature_extraction.text import TfidfVectorizer # для преобразования текста в вектор
from sklearn.linear_model import LogisticRegression # использование модели логистической регрессии
from sklearn.metrics import accuracy_score, classification_report # оценка производительности модели
from sklearn.pipeline import Pipeline # конвеер обработки данных

In [2]:
df=pd.read_csv('tweets.csv')
df.head()

,id,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,retweets,favorites,is_retweet
0,1340539111971516416,Rachel Roh,"La Crescenta-Montrose, CA",Aggregator of Asian American news; scanning di...,2009-04-08 17:52:46,405,1692,3247,False,2020-12-20 06:06:44,Same folks said daikon paste could treat a cyt...,['PfizerBioNTech'],Twitter for Android,0,0,False
1,1338158543359250433,Albert Fong,"San Francisco, CA","Marketing dude, tech geek, heavy metal & '80s ...",2009-09-21 15:27:30,834,666,178,False,2020-12-13 16:27:13,While the world has been on the wrong side of ...,NaN,Twitter Web App,1,1,False
2,1337858199140118533,eli🇱🇹🇪🇺👌,Your Bed,"heil, hydra 🖐☺",2020-06-25 23:30:28,10,88,155,False,2020-12-12 20:33:45,#coronavirus #SputnikV #AstraZeneca #PfizerBio...,"['coronavirus', 'SputnikV', 'AstraZeneca', 'Pf...",Twitter for Android,0,0,False
3,1337855739918835717,Charles Adler,"Vancouver, BC - Canada","Hosting ""CharlesAdlerTonight"" Global News Radi...",2008-09-10 11:28:53,49165,3933,21853,True,2020-12-12 20:23:59,"Facts are immutable, Senator, even when you're...",NaN,Twitter Web App,446,2129,False
4,1337854064604966912,Citizen News Channel,NaN,Citizen News Channel bringing you an alternati...,2020-04-23 17:58:42,152,580,1473,False,2020-12-12 20:17:19,Explain to me again why we need a vaccine @Bor...,"['whereareallthesickpeople', 'PfizerBioNTech']",Twitter for iPhone,0,0,False


In [3]:
sub_df=df.sample(frac=0.001)

In [4]:
sub_df.to_csv('subset.csv', index=False)

In [7]:
df1=pd.read_csv('googleplaystore.csv')
df1.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [16]:
df1.loc[df1['Rating']>4, 'Rating_Category']='High'
df1.loc[(df1['Rating']<=4) & (df1['Rating']>=2.5), 'Rating_Category']='Medium'
df1.loc[df1['Rating']<2.5, 'Rating_Category']='Low'
df1[['Rating','Rating_Category']]

,Rating,Rating_Category
0,4.1,High
1,3.9,Medium
2,4.7,High
3,4.5,High
4,4.3,High
...,...,...
10836,4.5,High
10837,5.0,High
10838,NaN,NaN
10839,4.5,High


In [18]:
df2=pd.read_csv('movie.csv')
df2.head()

,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1


In [20]:
X_train, X_test, y_train, y_test = train_test_split(
df2['text'], # текст рецензии - будет использоваться как входные данные
df2['label'], # метка классов (положительный или отрицательный )
test_size=0.2, # доля данных которая попадет в тест (20%)
random_state=42 # Зерно генератора случайных чисел для воспроизводимости результатов
)

In [21]:
pipeline = Pipeline([
('tfidf', TfidfVectorizer(stop_words='english')), # векторизация текста, с исключением стоп слов англ языка
('clf', LogisticRegression(max_iter=1000)) # модель логистической регресии с увелечинным колличеством итераций до 1000
])

In [22]:
#Обучение модели на обучающем наборе данных
pipeline.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer(stop_words='english')),
                ('clf', LogisticRegression(max_iter=1000))])

In [23]:
predictions = pipeline.predict(X_test) # предсказание модели на тестовых данных
accuracy = accuracy_score(y_test, predictions) # расчет точности модели
report = classification_report(y_test, predictions) # оценка качесвта предсказания

print(f"Точность модели {accuracy}")
print('Отчет по классификации')
print(report)

Точность модели 0.890125
Отчет по классификации
              precision    recall  f1-score   support

           0       0.90      0.88      0.89      3966
           1       0.88      0.90      0.89      4034

    accuracy                           0.89      8000
   macro avg       0.89      0.89      0.89      8000
weighted avg       0.89      0.89      0.89      8000

